# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [ ]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
#loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true')
#El arichivo estaba comprimido y use la , como delimitador
loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', sep=',', compression='gzip', encoding='latin-1')


<ipython-input-3-7c14ca5d6519>:3: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', sep=',', compression='gzip', encoding='latin-1')


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html

In [ ]:
# column_types
column_types = loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [ ]:
datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [ ]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [ ]:
# COdigo guardar
datos_dict.to_pickle('datos_dict.pkl')

In [ ]:
# Codigo para cargar
datos_dict_cargado = pd.read_pickle('datos_dict.pkl')
print(datos_dict_cargado)

                   feature                                        description
0           acc_now_delinq  The number of accounts on which the borrower i...
1     acc_open_past_24mths         Number of trades opened in past 24 months.
2               addr_state  The state provided by the borrower in the loan...
3                 all_util              Balance to credit limit on all trades
4               annual_inc  The self-reported annual income provided by th...
..                     ...                                                ...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...
151                    NaN                                                NaN
152                    NaN  * Employer Title replaces Employer Name for al...

[153 rows x 2 columns]


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [91]:
# Manejos de tipos 1
# Tu codigo aqui
# Eliminare las columnas que no contengan ningun dato ya que no van a proporcionar información importante y en el caso de member_id con el id es mas que suficiente para identificar y unnamed que no me da ninguna información
col_eliminar=['Unnamed','annual_inc_joint','dti_joint','verification_status_joint', 'revol_bal_joint','sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'member_id']
# Eliminar las columnas
# Verificar la existencia de cada columna antes de intentar eliminarla
for col in col_eliminar:
    if col in loans.columns:
        loans = loans.drop(columns=col)
# Imprimir el DataFrame resultante
loans = loans.loc[:, ~loans.columns.str.startswith('Unnamed')]
print(loans)

             id  loan_amnt  funded_amnt  funded_amnt_inv  term  int_rate  \
0      38098114    15000.0      15000.0          15000.0    60     12.39   
1      36805548    10400.0      10400.0          10400.0    36      6.99   
2      37842129    21425.0      21425.0          21425.0    60     15.59   
3      37612354    12800.0      12800.0          12800.0    60     17.14   
4      37662224     7650.0       7650.0           7650.0    36     13.66   
...         ...        ...          ...              ...   ...       ...   
99995  22454240     8400.0       8400.0           8400.0    36      9.17   
99996  11396920    10000.0      10000.0          10000.0    36     12.99   
99997   8556176    30000.0      30000.0          30000.0    60     20.99   
99998  24023408     8475.0       8475.0           8475.0    36     24.99   
99999  24023398    25000.0      25000.0          25000.0    36     10.15   

       installment grade sub_grade                        emp_title  ...  \
0          

In [92]:
print(loans.columns)

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag', 'settlement_status',
       'settlement_amount', 'settlement_percentage', 'settlement_term',
       'emp_length_numeric'],
      dtype='object', length=111)


In [94]:
# pymnt_plan lo voy a cambiar a valores booleanos para que sea mas fácil posteriormente
loans['pymnt_plan'] = loans['pymnt_plan'].replace({'n': False})
# Imprimir el DataFrame resultante
print(loans['pymnt_plan'])

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Name: pymnt_plan, Length: 100000, dtype: object


<ipython-input-94-9e80e2409ebb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['pymnt_plan'] = loans['pymnt_plan'].replace({'n': False})


In [95]:
# Para la columna terms los valores pueden ser 36 or 60, entonces le voy a quitar la palabra months
# Verificar si los valores son de tipo cadena y luego quitar " months" y convertir a int
loans['term'] = loans['term'].astype(str).str.replace(' months', '').astype(int)
# Imprimir el DataFrame resultante
print(loans['term'])


0        60
1        36
2        60
3        60
4        36
         ..
99995    36
99996    36
99997    60
99998    36
99999    36
Name: term, Length: 100000, dtype: int64


<ipython-input-95-04c471dc801f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['term'] = loans['term'].astype(str).str.replace(' months', '').astype(int)


In [96]:
# La columna emp_length podría traer problemas en un futuro a la hora de acceder, entonces  creamos otra con valores numéricos
#representando <1 year como 0 y 10+ years como 10
# Mapear las etiquetas existentes a valores numéricos
mapping_emp_length = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}

# Crear una nueva columna 'emp_length_numeric' con valores numéricos
loans['emp_length_numeric'] = loans['emp_length'].replace(mapping_emp_length)

# Imprimir el DataFrame resultante
print(loans[['emp_length', 'emp_length_numeric']])


      emp_length  emp_length_numeric
0      10+ years                  10
1        8 years                   8
2        6 years                   6
3      10+ years                  10
4       < 1 year                   0
...          ...                 ...
99995    2 years                   2
99996    3 years                   3
99997  10+ years                  10
99998  10+ years                  10
99999    3 years                   3

[100000 rows x 2 columns]


<ipython-input-96-7ed3619906b5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['emp_length_numeric'] = loans['emp_length'].replace(mapping_emp_length)


In [97]:
# vamos a cambair algunas columnas que son float pero no perderían nada de información si fueran int y así ocupan menos espacio
columnas_float = loans.select_dtypes(include='float64').columns.tolist()
# Imprimir las columnas de tipo float
print("Columnas de tipo float:", columnas_float)
#checamos cuales se pueden cambiar a int dependiendo de su descricpción
columnas_finales = ["delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec", "collections_12_mths_ex_med",
                    "policy_code", "acc_now_delinq", "tot_coll_amt", "acc_open_past_24mths", "chargeoff_within_12_mths",
                    "delinq_amnt", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
                    "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats", "num_bc_tl",
                    "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats",
                    "num_tl_30dpd", "num_tl_90g_dpd_24m", "num_tl_op_past_12m", "pub_rec_bankruptcies", "tax_liens"]

loans[columnas_finales] = loans[columnas_finales].astype('int')

# Imprimir el DataFrame resultante
print(loans[columnas_finales])

Columnas de tipo float: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'fico_range_low', 'fico_range_high', 'mths_since_last_delinq', 'mths_since_last_record', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'mths_since_last_major_derog', 'tot_cur_bal', 'total_rev_hi_lim', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_tl_120dpd_2m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'deferral_term', 'hardship_amount', 'hardship_length', 'hardship_dpd', 'orig_projected_additional_accrued_interest', 'hardship_payoff

<ipython-input-97-41ec497738ec>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans[columnas_finales] = loans[columnas_finales].astype('int')


In [98]:
descripcion_hardship_flag = loans['hardship_flag'].describe()
print(descripcion_hardship_flag)

count     100000
unique         1
top         True
freq      100000
Name: hardship_flag, dtype: object


In [99]:
#hardship_flag	Flags whether or not the borrower is on a hardship plan las N a valores booleanos porque es mas facil trabajar con boolean
# Actualizar los valores en la columna 'hardship_flag'
loans.loc[loans['hardship_flag'] == 'N', 'hardship_flag'] = False
loans.loc[loans['hardship_flag'] != 'N', 'hardship_flag'] = True
# Imprimir el DataFrame resultante
print(loans['hardship_flag'])

#igual que debt_settlement_flag
# Actualizar los valores en la columna 'debt_settlement_flag'
loans.loc[loans['debt_settlement_flag'] == 'N', 'debt_settlement_flag'] = False
loans.loc[loans['debt_settlement_flag'] != 'N', 'debt_settlement_flag'] = True

# Imprimir el DataFrame resultante
print(loans['debt_settlement_flag'])


0        True
1        True
2        True
3        True
4        True
         ... 
99995    True
99996    True
99997    True
99998    True
99999    True
Name: hardship_flag, Length: 100000, dtype: bool
0        True
1        True
2        True
3        True
4        True
         ... 
99995    True
99996    True
99997    True
99998    True
99999    True
Name: debt_settlement_flag, Length: 100000, dtype: object


In [100]:
# Recrear la tabla column_types con los nuevos tipos de datos
column_types = pd.DataFrame(loans.dtypes, columns=['type']).reset_index()
column_types.rename(columns={'index': 'column_name'}, inplace=True)

# Mostrar la nueva tabla de tipos de datos
print("Nueva tabla de tipos de datos:")
print(column_types)

Nueva tabla de tipos de datos:
               column_name     type
0                       id    int64
1                loan_amnt  float64
2              funded_amnt  float64
3          funded_amnt_inv  float64
4                     term    int64
..                     ...      ...
106      settlement_status   object
107      settlement_amount  float64
108  settlement_percentage  float64
109        settlement_term  float64
110     emp_length_numeric    int64

[111 rows x 2 columns]


## Manejo de NaNs o missings

Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`.


De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [101]:
import numpy as np

# Inicializar el diccionario de imputación
imput_dict = {}

# Reemplazar espacios en blanco por NaN
loans.replace('', np.nan, inplace=True)

def imputacion(dataframe):
    for column_name in dataframe.columns:
        if dataframe[column_name].dtype == 'int64':
            # Imputar NaN con la media para columnas de tipo int64
            mean_value = int(dataframe[column_name].mean())
            dataframe[column_name].fillna(mean_value, inplace=True)
            imput_dict[column_name] = {'estrategia': 'media', 'valor': mean_value}
        elif dataframe[column_name].dtype == 'float64':
            # Imputar NaN con la media para columnas de tipo float64
            mean_value = dataframe[column_name].mean()
            dataframe[column_name].fillna(mean_value, inplace=True)
            imput_dict[column_name] = {'estrategia': 'media', 'valor': mean_value}
        elif dataframe[column_name].dtype == 'object':
            # Imputar NaN con la moda para columnas de tipo object
            mode_value = str(dataframe[column_name].mode().iloc[0])  # Obtener el primer valor de la moda
            dataframe[column_name].fillna(mode_value, inplace=True)
            imput_dict[column_name] = {'estrategia': 'moda', 'valor': mode_value}
        elif dataframe[column_name].dtype == 'string':  # Corregir dtype 'string' a 'object'
            # Imputar NaN con la moda para columnas de tipo object
            mode_value = str(dataframe[column_name].mode().iloc[0])  # Obtener el primer valor de la moda
            dataframe[column_name].fillna(mode_value, inplace=True)
            imput_dict[column_name] = {'estrategia': 'moda', 'valor': mode_value}

        # Eliminar columnas con más del 80% de NaN
        if dataframe[column_name].isnull().mean() >= 0.8:
            dataframe.drop(column_name, axis=1, inplace=True)

    return dataframe

# Aplicar la función de imputación al DataFrame 'loans'
loans = imputacion(loans)

# Imprimir el diccionario de imputación resultante
print("Diccionario de imputación:")
print(imput_dict)


<ipython-input-101-0faff63a546e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans.replace('', np.nan, inplace=True)
<ipython-input-101-0faff63a546e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name].fillna(mean_value, inplace=True)
<ipython-input-101-0faff63a546e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name].fillna(mean_value, inplace=True)
<ipython-input-101-0faff63a546e>:24: Se

Diccionario de imputación:
{'id': {'estrategia': 'media', 'valor': 30299954}, 'loan_amnt': {'estrategia': 'media', 'valor': 14886.93}, 'funded_amnt': {'estrategia': 'media', 'valor': 14886.93}, 'funded_amnt_inv': {'estrategia': 'media', 'valor': 14883.9105}, 'term': {'estrategia': 'media', 'valor': 43}, 'int_rate': {'estrategia': 'media', 'valor': 13.2780734}, 'installment': {'estrategia': 'media', 'valor': 437.3318244000001}, 'grade': {'estrategia': 'moda', 'valor': 'C'}, 'sub_grade': {'estrategia': 'moda', 'valor': 'C1'}, 'emp_title': {'estrategia': 'moda', 'valor': 'Teacher'}, 'emp_length': {'estrategia': 'moda', 'valor': '10+ years'}, 'home_ownership': {'estrategia': 'moda', 'valor': 'MORTGAGE'}, 'annual_inc': {'estrategia': 'media', 'valor': 74689.2425825}, 'verification_status': {'estrategia': 'moda', 'valor': 'Source Verified'}, 'loan_status': {'estrategia': 'moda', 'valor': 'Fully Paid'}, 'pymnt_plan': {'estrategia': 'moda', 'valor': 'False'}, 'desc': {'estrategia': 'moda', 'va

<ipython-input-101-0faff63a546e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name].fillna(mode_value, inplace=True)
<ipython-input-101-0faff63a546e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name].fillna(mode_value, inplace=True)
<ipython-input-101-0faff63a546e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column_name].fillna(mode_value, inplace=True)
<ipython-input-101-0fa

### Codigo para salvar y cargar JSONs

In [102]:
import json
# Guardar el diccionario en un archivo JSON
with open('imput_dict.json', 'w') as archivo_json:
    json.dump(imput_dict, archivo_json)

In [103]:
# Cargar el diccionario desde el archivo JSON
with open('imput_dict.json', 'r') as archivo_json:
    json_dict = json.load(archivo_json)
# Imprimir el diccionario cargado
print(json_dict)


{'id': {'estrategia': 'media', 'valor': 30299954}, 'loan_amnt': {'estrategia': 'media', 'valor': 14886.93}, 'funded_amnt': {'estrategia': 'media', 'valor': 14886.93}, 'funded_amnt_inv': {'estrategia': 'media', 'valor': 14883.9105}, 'term': {'estrategia': 'media', 'valor': 43}, 'int_rate': {'estrategia': 'media', 'valor': 13.2780734}, 'installment': {'estrategia': 'media', 'valor': 437.3318244000001}, 'grade': {'estrategia': 'moda', 'valor': 'C'}, 'sub_grade': {'estrategia': 'moda', 'valor': 'C1'}, 'emp_title': {'estrategia': 'moda', 'valor': 'Teacher'}, 'emp_length': {'estrategia': 'moda', 'valor': '10+ years'}, 'home_ownership': {'estrategia': 'moda', 'valor': 'MORTGAGE'}, 'annual_inc': {'estrategia': 'media', 'valor': 74689.2425825}, 'verification_status': {'estrategia': 'moda', 'valor': 'Source Verified'}, 'loan_status': {'estrategia': 'moda', 'valor': 'Fully Paid'}, 'pymnt_plan': {'estrategia': 'moda', 'valor': 'False'}, 'desc': {'estrategia': 'moda', 'valor': ' '}, 'purpose': {'es